# <p style="text-align: center;">  University College London </p>
## <p style="text-align: center;"> Python Assignment  </p>
### <p style="text-align: center;"> Maria Lavrovskaya </p>
#### <p style="text-align: center;"> 29.03.2021 </p>


This report is organized as follows: 
* The exercises and theory are written in Markdown
* The code is written in Python as assignment suggests, the code explanations are written as comments
* This report includes interesting observations related to Python programming which will be under <mark>Python Side Notes</mark>
* The visualizations are done with BokehJS which allows to do interactive plotting. The visualizations might not be visible if the kernel is disconnected. If upon marking this assignment the visualizations are not visible, the cells that are marked should be rerun (<mark>MARKED</mark>). If one does not have JupyterLab and BokehJS installed in IPython Notebook, this can easily done with three commands: 
 1. Use this command to install Nodejs: ```pip install nodejs ``` or ```conda install nodegs ```
 2. Use this command to install Bokeh:```pip install bokeh ``` or ```conda install bokeh ```
 3. First install jupyterlab-manager with this command: 
     ```jupyter labextension install @jupyter-widgets/jupyterlab-manager```
 4. Then install the jupyter_bokeh extension as follows:
     ``` jupyter labextension install @bokeh/jupyter_bokeh ```
 
If the issues are experienced installing the packages, we have also created two recordings, which are provided in the folder that explain the plots.

***

In [ ]:
import numpy as np
np.set_printoptions(suppress=True)

For this assignment we consider the linear system 
\begin{equation}
A\textbf{x}=\textbf{b} \text{ such that } A = 
\begin{bmatrix}
0 & 3 & -1 & 8\\
-1 & 11 & -1 & 3\\
2 & -1 & 10 & -1\\
10 & -1 & 2 & 0
\end{bmatrix}
\textbf{,  b = }
\begin{bmatrix}
15\\
25\\
-11\\
6
\end{bmatrix}
\text{, where } \textbf{x} = (x_{1}, x_{2}, x_{3}, x_{4})^{T}
\end{equation}

In [3]:
A = np.array([[0., 3., -1., 8.], 
      [-1., 11., -1., 3.], 
      [2., -1., 10.,-1. ], 
      [10., -1., 2., 0.]])

b = np.array([15. , 25., -11., 6.])

***

### Exercise 1. Write a program to test if the matrix A is strictly diagonally dominant. 

An **M** with entries $\textbf{M}_{ij}$ is *strictly diagonal dominant* if
\begin{equation*}
|M_{ij}| > \sum_{j \neq i}|M_{ij}|
\end{equation*}

In [4]:
# Exercise 1
def diagonal_dominance(A):
    """ Function checks if matrix is stricly diagonal dominant
    Parameters 
    ----------
    matrix: numpy.array
            Square matrix to check
    Returns 
    ----------
    None; prints if the matrix is strictly diagonal dominant / not
    
    """
    elem_diagonals = [A[i][i] for i in range(len(A))]
    rows_sums = [np.sum([np.abs(A[i][i-1]),np.abs(A[i][i-2]),np.abs(A[i][i-3])]) for i in range(len(A))]
    if np.all(elem_diagonals > rows_sums):
        print('Matrix is strictly diagonal dominant')
    else: 
        print('Matrix is not diagonal dominant')
    return None 

In [5]:
diagonal_dominance(A)

Matrix is not diagonal dominant


The initial matrix $A$ is not strictly diagonal dominant. It can be shown that if the matrix is strictly diagonally dominant, the algorithm can not encounter a zero pivot. Diagonal dominance will be important for the convergence of the Gauss-Seidel and SOR methods (as described later on) and will be checked again once row exchange is completed. 

***

### Exercise 2. LU Decomposition: Write a program to obtain a solution of system above using both methods of Doolittle and Crout. Your program in each case should list L and U; followed by the solution for x

We are given $n$ by $n$ matrix A and the $n$ by 1 column vector $\textbf{b}$. We look for the solution vector $\textbf{x}$. We solve the equations by simplifying them - eliminating $x_{1}$ from $n-1$ equations to get a smaller system $A_{2}\textbf{x}_{2}=\textbf{b}_{2}$ of size $n-1$. Eventually we reach the 1 by 1 system $A_{n}x_{n}=b_{n}$. The point is to see those elimination steps in terms of rank 1 matrices. Every step (from A to $A_{2}$ and eventually $A_{n}$ removes a matrix $lu^{*}$. Then the original A is the sum of those rank 1 matrices. This sum is exactly the great factorization $A=LU$ into lower and upper triangular matrices L and U [<sup>3</sup>](#fn3). 


Thus, here is a whole idea briefly outlined: 

Column 1. Use equation 1 to create zeros below the first pivot. Note, pivots can not be zero!

Column 2. Use the new equation 2 to create zeros below the second pivot. 

Columns 3 to n. Keep going to find the upper triangular U: n pivots on its diagonal. 

These steps explain the Doolittle's method - that is if $l_{ii} = 1 \forall 1 \leq i \leq n$. There is another method called Crout's method, which is "flipped" - that is $u_{ii} = 1 \forall 1 \leq i \leq n$.

This matrix $A$ has first zero pivot. As we need to divide by these numbers, we have already said that we require non-zero pivots. Therefore, we use a permutation matrix. The whole idea of permutation matrix is to change the rows of the $A$ so the first pivot is no longer zero and it is a good practice to choose the largest number to be the first pivot. 

Once the permutation matrix is used, both sides of the system $A\textbf{x}=\textbf{b}$ should be multiplied by $P$, that is our permutation matrix.

In [6]:
def permutation_matrix(A):
    """ Function checks if the first pivot of given matrix is 0 and then returns permutation matrix, otherwise None
    Parameters 
    ----------
    matrix: numpy.array
            Matrix which is checked 
    Returns 
    ----------
    I: numpy.array
           Permutation matrix
    """
    
    if A[0][0] == 0:
        I = np.identity(4)
        exchanged_row = max(range(len(A)), key=lambda candidate: abs(A[candidate][0]))
        if 0!=exchanged_row:
            I[[0, exchanged_row], :] = I[[exchanged_row, 0], :] #selecting the submatrix of rows 
        return I
    else:
        print('Permutation is not needed, since the first pivot is not 0')
        return None

In [7]:
# We determine the permutation matrix
P = permutation_matrix(A)
# We multiply both sides of the system by the permutation matrix
A = np.dot(P,A)
b = np.dot(P,b)
print('A matrix after multiplying by P matrix', '\n', A, '\n', 'b vector after multiplying by P', '\n', b)

A matrix after multiplying by P matrix 
 [[10. -1.  2.  0.]
 [-1. 11. -1.  3.]
 [ 2. -1. 10. -1.]
 [ 0.  3. -1.  8.]] 
 b vector after multiplying by P 
 [  6.  25. -11.  15.]


In [8]:
# Exercise 2
def ludecomposition(matrix, style='Doolittle'):
    """ Function does LU factorisation
    Parameters 
    ----------
    matrix: numpy.array
            Matrix for the decomposition
    style: string
            "Crout" if the decomposition should be done with Crout's method
            Any other string will perform decomposotion with Doolittle's method. This is done by Default.
    Returns 
    ----------
    lower: numpy.array
           Lower triangular matrix
    upper: numpy.array
           Upper triangular matrix
    """
    lower = np.zeros((4,4))
    upper = np.zeros((4,4))
    if style == 'Crout': 
        for pivot in range(len(matrix)):
            upper[pivot,:] = matrix[pivot,:] / matrix[pivot,pivot] #divide the first raw of reduced matrix by pivot
            lower[:,pivot] = matrix[:, pivot] #write the elements of pivot column as the elements of lower matrix
            matrix= matrix - lower[:, pivot:] @ upper[pivot:, :] 
    else:
        for pivot in range(len(matrix)):
            lower[:,pivot] = matrix[:, pivot] / matrix[pivot][pivot] #divide the first column of reduced matrix by pivot (1)
            upper[pivot,:] = matrix[pivot] #write the elements of pivot row as the element of upper matrix (2)
            matrix = matrix - lower[:, pivot:] @ upper[pivot:, :] #remove the matrix (created on every iteration (1) and (2)) to find remaining matrix
#             matrix = matrix[pivot, pivot:] - lower[:, pivot:] @ upper[pivot:, :]
    return lower, upper


<mark>Python Side Notes</mark>: While doing the exercise, we stumbled upon an interesting behaviour we were not aware about. If we use the assignment operator ```-=``` in the line ```matrix= matrix - lower[:, pivot:] @ upper[pivot:, :]  ```, the entries of original matrix are changed, not the values of the local variable within scope of function call. This is happening because in Python the variables are passed by reference, thus we have two names (A and matrix) referring to the same original array. Thus, once we change the entries of matrix with assignment operator, we change the original array. ```matrix= matrix - ...  ``` allows to create a copy and is used only inside the function call and does not modify otiginal array.

***

Once the decomposition is found we then solve the original system by solving two smaller problems. The tridiagonal system is written in the form

\begin{equation*}
\textbf{My = (LU)y = L(Uy) = p}
\end{equation*}

We introduce an intermediate vector $\textbf{z}$. $z_{i}$ for whether lower triangular matrix is found with Doolittle's method or Crout's method can be found by forward substitution:
\begin{equation*}
z_{i}=\frac{( b_{i} - \sum_{j=1}^{i-1}l_{ij}z_{j})}{l_{ii}} 
\end{equation*}

Last division is only necessary if decomposition is obtained with Crout's method. If LU factorisation is obrained with Doolittle's method, $l_{ii}$ will be one. 

In [9]:
def forward_substitution(lower, b):
    """ Function finds the intermediate vector z by forward substitution
    Parameters 
    ----------
    lower: numpy.array
        Lower tridiagonal matrix from LU    
    b: numpy.array
        b vector from linear system
    Returns 
    ----------
    z: numpy.array
           intermediate vector
    upper: numpy.array
           Upper triangular matrix
    """
    z = np.zeros(4)
    for i in range(0,len(b)):
        z[i] = (b[i] - np.dot(lower[i,:i], z[:i])) / lower[i,i] #this last division is only necessary for the Crout case, for Doolittle lower[i,i] = 1  
    return z

Having obtained $\textbf{z}$ we find $\textbf{y}$ by solving $\textbf{Uy=z}$. 

\begin{equation*}
y_{i}=\frac{( z_{i} - \sum_{j=i+1}^{n}u_{ij}x_{j})}{u_{ii}} 
\end{equation*}

Last division is only necessary if decomposition is obtained with Doolittle method. If LU factorisation is obrained with Crout's method, $u_{ii}$ will be one. 

In [10]:
def backward_substitution(upper, z):
    """ Function finds the solution vector y
    Parameters 
    ----------
    upper: numpy.array
        Upper triangular matrix from LU    
    z: numpy.array
        Intermediate vector z found by forward-substituion
    Returns 
    ----------
    y: numpy.array
           solution vector
    """
    y = np.zeros(4)
    for i in range(len(upper)-1, -1, -1): #range now goes from n to 1
        y[i] = (z[i] - np.dot(upper[i, i:], y[i:])) / upper[i,i] #this last division is only necessary for the Doolittle case, for Crout upper[i,i] = 1  
    return y

We now show that the solution vector of the system using both Doolittle's and Crout's methods.

In [11]:
# Doolittle method
L, U = ludecomposition(A, style='Doolittle')
z = forward_substitution(L, b)
y = backward_substitution(U, z)
print('Lower triangular matrix:',  '\n' ,L, '\n',
      'Upper triangular matrix:', '\n', U, '\n',
      'Intermediate vector z:','\n', z, '\n', 
      'Solution vector:', y)

Lower triangular matrix: 
 [[ 1.          0.          0.          0.        ]
 [-0.1         1.          0.          0.        ]
 [ 0.2        -0.0733945   1.          0.        ]
 [ 0.          0.27522936 -0.08173077  1.        ]] 
 Upper triangular matrix: 
 [[10.         -1.          2.          0.        ]
 [ 0.         10.9        -0.8         3.        ]
 [ 0.          0.          9.5412844  -0.77981651]
 [ 0.         -0.          0.          7.11057692]] 
 Intermediate vector z: 
 [  6.          25.6        -10.32110092   7.11057692] 
 Solution vector: [ 1.  2. -1.  1.]


In [12]:
# Crout method
L, U = ludecomposition(A, style='Crout')
z = forward_substitution(L, b)
y = backward_substitution(U, z)
print('Lower triangular matrix:',  '\n' ,L, '\n',
      'Upper triangular matrix:', '\n', U, '\n',
      'Intermediate vector z:','\n', z, '\n', 
      'Solution vector:', y)

Lower triangular matrix: 
 [[10.          0.          0.          0.        ]
 [-1.         10.9         0.         -0.        ]
 [ 2.         -0.8         9.5412844   0.        ]
 [ 0.          3.         -0.77981651  7.11057692]] 
 Upper triangular matrix: 
 [[ 1.         -0.1         0.2         0.        ]
 [ 0.          1.         -0.0733945   0.27522936]
 [ 0.          0.          1.         -0.08173077]
 [ 0.          0.          0.          1.        ]] 
 Intermediate vector z: 
 [ 0.6         2.34862385 -1.08173077  1.        ] 
 Solution vector: [ 1.  2. -1.  1.]


<mark>Python Side Notes</mark>: Minus before 0 L[1,3] in Crout's method is not insignificant, because it is a very-very small number that rounds to 0 and does not effect any futher computations involved in finding solution vector. Presumably, this is happening due to the rounding error (the same stands for -0. in U[3,1] in Doolittle's method). 

***

### Exercise 3. Write a program to solve the system above. Use an initial guess $x^{(0)}=(0,0,0,0)$ and then repeat with $x^{(0)}=(1,1,1,1)$ ;examine if one converges quicker than the other with consistency to four decimal places. At east iteration step the solution vector should be printed together with use of the $l_{\infty}$ norm.

The Gauss-Seidel method will converge to the solution of the original problem, 

\begin{equation*}
\lim_{k \rightarrow \infty} \hat v_{n}^{(k)}= v_{n}
\end{equation*}

if a matrix $M$ is strictly diagonally dominant. This property has been explained before.

We should note, that in the beginning of this report we show that matrix $A$ is not strictly diagonally dominant. We then exchanged rows multiplying by permutation matrix $P$. We now check if $PA$ is indeed strictly diagonally dominant.

In [13]:
diagonal_dominance(A) #A have been reassigned to A = np.dot(P,A) before

Matrix is strictly diagonal dominant


We now showed that $A$ is strictly diagonally dominant and therefore we expect the Gauss-Seidel method to converge to the actual solution vector.

For Gauss-Seidel we have [<sup>5</sup>](#fn5):

\begin{equation*}
x_{i}^{k+1}=\frac{1}{a_{ii}} \big[-\sum_{j=1}^{i-1} \big(a_{i j}x_{j}^{k+1} \big ) - \sum_{j=i+1}^{n} \big (a_{i j}x_{j}^{(k)} \big )+b_{i} \big ], \
(i = 1, ...n)
\end{equation*}

**Error**. The absolute error ($\bf{e^{k}}$):

\begin{equation*}
||\hat x - x||_{\infty},
\end{equation*}
$\text{ where } \bf{\hat x} \text{ is an approximation and } \bf{x} \text{ is a true answer.}$

And the relative error: 
\begin{equation*}
\frac{||\hat x - x||_{\infty}}{||x||_{\infty}}
\end{equation*}

Note, $\bf{x^{k}}$$\rightarrow$ $\bf{0}$ as $ k \rightarrow \infty$ for all $\bf{e^{0}}$ if and only if $ ||\bf{e^{k}}|| \rightarrow 0$ [<sup>1</sup>](#fn1)


One big advantage of iterative methods is that we can control how accurate we want our solution, for example, if we want our solution to be $\varepsilon$-accurate, then in principle we can stop as soon as
\begin{equation*}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\frac{\norm{\underline{x}^{(k+1)} - \underline{x}^{(k)}}_{\infty}}{\norm{\underline{x}^{(k)}}_{\infty}} < \varepsilon\\
\varepsilon \text{ is the specified tolerance (>0) and } \norm{\underline{x}}_{\infty} = \max_{1 \leq i \leq n} |x_{i}|
\end{equation*}

Since the exercise requires the consistency to four decimal points, we set $\varepsilon = 1e-4$

In [14]:
# Exercise 3, 4
def gauss_seidel(A, b, x0, tol = 1e-4, maxIter = 10, omega = 1):
    """ Function solves system of linear equations Ax=b (A and b must have the same number of rows)
    Parameters 
    ----------
    A: numpy.array
        Coefficient matrix
    b: numpy.array
        A vector
    tol: float
        Specified tolerance
    maxIter: int
        The maximum number of iterations
    omega: int
        Relaxation factor. a) Omega = 1 - Gauss-Seidel method 
                           b) For 0 < omega < 1 - under-relaxation methods (used when GS is not convergent)
                           c) For omega > 1 - over-relaxation methods (used when GS is convergent to accelerate converg.)
        
            
    Returns 
    ----------
    x: numpy.array
        Solution vector
    x_history: numpy.array
        x in every iteration
    norms: numpy.array
        The convergence criterion
    
    """
    x = x0.copy()
    iterations = 0
    stopCond = 1
    norms = []
    x_history = np.array([x0])
    while (iterations < maxIter) and (stopCond > tol): 
        iterations += 1
        previous_x = x.copy()
        for i in range(len(b)):
            x[i] = ((-(np.dot(A[i,:i], x[:i]) + np.dot(A[i,i+1:], x[i+1:])) + b[i]) 
                    *omega/A[i,i]) + np.dot(x[i], (1-omega))
        x_history = np.append(x_history, [x], axis = 0)
        stopCond = np.linalg.norm(x - previous_x, ord=np.inf) / np.linalg.norm(previous_x, ord=np.inf)
#         stopCond = np.around(stopCond, decimals=4)
        norms.append(stopCond)
        print('Iteration:', iterations,'\n', 'Solution vector:', x, '\n', 'Inf norm:', stopCond)
    return x, np.array(norms), x_history, iterations

<mark>MARKED</mark> **Let's now calculate the results, plot them and investigate covergence. If the plot down below is not generated, one should simply rerun the functions written below. Interactive plots are integrated with JS so they might not be visible once the notebook is disconnected from kernel.**

In [57]:
x0 = np.zeros(4)
x00, norms00, xhistory00, iterations = gauss_seidel(A,b, x0)

Iteration: 1 
 Solution vector: [ 0.6         2.32727273 -0.98727273  0.87886364] 
 Inf norm: inf
Iteration: 2 
 Solution vector: [ 1.03018182  2.03693802 -1.0144562   0.98434122] 
 Inf norm: 0.18484375000000003
Iteration: 3 
 Solution vector: [ 1.00658504  2.00355502 -1.00252738  0.99835095] 
 Inf norm: 0.016388814658793216
Iteration: 4 
 Solution vector: [ 1.00086098  2.00029825 -1.00030728  0.99984975] 
 Inf norm: 0.002856953090344185
Iteration: 5 
 Solution vector: [ 1.00009128  2.00002134 -1.00003115  0.9999881 ] 
 Inf norm: 0.0003847917873479008
Iteration: 6 
 Solution vector: [ 1.00000836  2.00000117 -1.00000275  0.99999922] 
 Inf norm: 4.145786992800611e-05


<ipython-input-14-2bf733c1a108>:42: RuntimeWarning: divide by zero encountered in double_scalars
  stopCond = np.linalg.norm(x - previous_x, ord=np.inf) / np.linalg.norm(previous_x, ord=np.inf)


In [18]:
x0 = np.ones(4)
x11, norms11, xhistory11,iterations = gauss_seidel(A,b, x0)

Iteration: 1 
 Solution vector: [ 0.5         2.13636364 -0.88636364  0.96306818] 
 Inf norm: 1.8863636363636362
Iteration: 2 
 Solution vector: [ 0.99090909  2.01957645 -0.99991736  0.99266916] 
 Inf norm: 0.22978723404255322
Iteration: 3 
 Solution vector: [ 1.00194112  2.0021833  -1.00090298  0.99906839] 
 Inf norm: 0.008612275598771235
Iteration: 4 
 Solution vector: [ 1.00039893  2.00020825 -1.00015212  0.99990289] 
 Inf norm: 0.0009864457246365598
Iteration: 5 
 Solution vector: [ 1.00005125  2.00001731 -1.00001823  0.99999123] 
 Inf norm: 0.00017381979942646743
Iteration: 6 
 Solution vector: [ 1.00000538  2.00000122 -1.00000183  0.99999931] 
 Inf norm: 2.293582118326794e-05


In [19]:
# Necessary so the error won't be shown 
from bokeh.io import reset_output
reset_output()

In [21]:
# Relative errors for x0 = (0,0,0,0).T and iterations
solution = np.linalg.solve(A, b)
errors00 = np.array([xhistory00[i]-solution for i in range(len(xhistory00))])
relative_errors_00 = [np.linalg.norm(errors00[i], ord=np.inf) / np.linalg.norm(solution, ord=np.inf) for i in range(len(errors00))]
                                                                                                         
iterations00 = [i for i in range(1,len(xhistory00))]


In [38]:
iterations00

[1, 2, 3, 4, 5, 6]

In [22]:
# Relative errors for x0 = (1,1,1,1).T and iterations
errors11 = np.array([xhistory11[i]-solution for i in range(len(xhistory11))])
relative_errors_11 = [np.linalg.norm(errors11[i], ord=np.inf) / np.linalg.norm(solution, ord=np.inf) for i in range(len(errors11))]
                                                                                                         
iterations11 = [i for i in range(1,len(xhistory11))]


In [23]:
import pandas as pd
# Dataframes for norms x0 = (0,0,0,0).T
d = {'iterations':iterations00,'Norms00': norms00, 'x0':xhistory00[1:, 0],'x1':xhistory00[1:, 1], 'x2':xhistory00[1:, 2], 'x3':xhistory00[1:, 3] }
df_xhistory00 = pd.DataFrame(data=d)


# Dataframe for errors x0 = (0,0,0,0).T
d = {'iterations':iterations00,'Relative Errors 00': relative_errors_00[1:], 'Error_x0':errors00[1:, 0],'Error_x1':errors00[1:, 1], 'Error_x2':errors00[1:, 2], 'Error_x3':errors00[1:, 3] }
df_errors00 = pd.DataFrame(data=d)
df_errors00

,iterations,Relative Errors 00,Error_x0,Error_x1,Error_x2,Error_x3
0,1,0.200000,-0.400000,0.327273,0.012727,-1.211364e-01
1,2,0.018469,0.030182,0.036938,-0.014456,-1.565878e-02
2,3,0.003293,0.006585,0.003555,-0.002527,-1.649054e-03
3,4,0.000430,0.000861,0.000298,-0.000307,-1.502535e-04
4,5,0.000046,0.000091,0.000021,-0.000031,-1.189674e-05
5,6,0.000004,0.000008,0.000001,-0.000003,-7.831352e-07


In [24]:
# Dataframe for norms x(0) = (1,1,1,1).T
d = {'iterations':iterations00,'Norms11': norms11, 'x0':xhistory11[1:, 0],'x1':xhistory11[1:, 1], 'x2':xhistory11[1:, 2], 'x3':xhistory11[1:, 3] }
df_xhistory11 = pd.DataFrame(data=d)
df_xhistory11

# Dataframe for errors x(0) = (1,1,1,1).T
d = {'iterations':iterations11,'Relative Errors 11': relative_errors_11[1:], 'Error_x0':errors11[1:, 0],'Error_x1':errors11[1:, 1], 'Error_x2':errors11[1:, 2], 'Error_x3':errors11[1:, 3] }
df_errors11 = pd.DataFrame(data=d)
df_errors11

,iterations,Relative Errors 11,Error_x0,Error_x1,Error_x2,Error_x3
0,1,0.250000,-0.500000,0.136364,0.113636,-3.693182e-02
1,2,0.009788,-0.009091,0.019576,0.000083,-7.330837e-03
2,3,0.001092,0.001941,0.002183,-0.000903,-9.316086e-04
3,4,0.000199,0.000399,0.000208,-0.000152,-9.710965e-05
4,5,0.000026,0.000051,0.000017,-0.000018,-8.771549e-06
5,6,0.000003,0.000005,0.000001,-0.000002,-6.877297e-07


In [25]:
# Creating data structures for the Bokeh internal use
from bokeh.models import ColumnDataSource, NumeralTickFormatter
bokeh_xhistory00 = ColumnDataSource(df_xhistory00)
bokeh_xhistory11 = ColumnDataSource(df_xhistory11)

bokeh_errors00 = ColumnDataSource(df_errors00)
bokeh_errors11 = ColumnDataSource(df_errors11)

In [39]:
# Creating plotting objects
from bokeh.plotting import figure, output_file, show
norms = figure(
                  plot_height=300, plot_width=600,
                  title='Convergence criterion',
                  x_axis_label='Iterations', y_axis_label='Convergence Criterion',
                  toolbar_location=None)
norms.line(iterations00, norms00, legend_label='x0 = (0,0,0,0).T', line_color="blue", line_width=2, muted_alpha=0)
norms.line(iterations11, norms11, legend_label='x0 = (1,1,1,1).T', line_color="orange", line_width=2, muted_alpha=0)
norms.circle(x='iterations',
           y='Norms00',
           source = bokeh_xhistory00,
           color='royalblue',
             fill_color="white",
           selection_color='deepskyblue',
           nonselection_color='lightgray',
           legend_label='x0 = (0,0,0,0).T',
            muted_alpha=0,
           nonselection_alpha=0.3)
norms.circle(x='iterations',
           y='Norms11',
           source = bokeh_xhistory11,
           color='orange',
             fill_color="white",
           selection_color='yellow',
           nonselection_color='lightgray',
           legend_label='x0 = (1,1,1,1).T',
           muted_alpha = 0,
           nonselection_alpha=0.3)



errors_plot = figure(
                  plot_height=300, plot_width=600,
                  title='Relative Errors (L-infinity norm)',
                  x_axis_label='Iterations', y_axis_label='Relative Error',
                  toolbar_location=None)
errors_plot.line(iterations00, relative_errors_00[1:], legend_label='x0 = (0,0,0,0).T', line_color="blue", line_width=2, muted_alpha=0)
errors_plot.line(iterations11, relative_errors_11[1:], legend_label='x0 = (1,1,1,1).T', line_color="orange", line_width=2, muted_alpha=0)
errors_plot.circle(x='iterations',
           y='Relative Errors 00',
           source = bokeh_errors00,
           color='royalblue',
           fill_color="white",
           selection_color='deepskyblue',
           nonselection_color='lightgray',
           legend_label='x0 = (0,0,0,0).T',
            muted_alpha=0,
           nonselection_alpha=0.3)
errors_plot.circle(x='iterations',
           y='Relative Errors 11',
           source = bokeh_errors11,
           color='orange',
                   fill_color="white",
           selection_color='yellow',
           nonselection_color='lightgray',
           legend_label='x0 = (1,1,1,1).T',
           muted_alpha = 0,
           nonselection_alpha=0.3)


GlyphRenderer(id='1653', ...)

In [40]:
# To mute the legend when needed
norms.legend.click_policy = 'hide'
norms.legend.click_policy = 'mute'

errors_plot.legend.click_policy = 'hide'
errors_plot.legend.click_policy = 'mute'

In [41]:
# For Hoovering
from bokeh.models import HoverTool


# Format the tooltip
tooltips_norms = [
            ('x0','@x0'),
            ('x1','@x1'),
            ('x2','@x2'),
            ('x3','@x3'),
           ]

tooltips_errors = [
            ('Error in x0','@Error_x0'),
            ('Error in x1','@Error_x1'),
            ('Error in x2','@Error_x2'),
            ('Error in x3','@Error_x3'),
           ]

# Add the HoverTool to the figure
norms.add_tools(HoverTool(tooltips=tooltips_norms))
errors_plot.add_tools(HoverTool(tooltips=tooltips_errors))

In [42]:
from bokeh.io import output_file, output_notebook
from bokeh.models.widgets import Tabs, Panel

# Create two panels, one for each conference
norms_panel = Panel(child=norms, title='Convergence Criterion')
errors_panel = Panel(child=errors_plot, title='Errors')

# Assign the panels to Tabs
tabs = Tabs(tabs=[norms_panel, errors_panel])

In [98]:
output_notebook()
show(tabs)

Loading BokehJS ...

Note, the following visualisation is interactive. The 'Convergence Criterion' plot shows the iteration on the x-axis and the quotient $\frac{\norm{\underline{x}^{(k+1)} - \underline{x}^{(k)}}_{\infty}}{\norm{\underline{x}^{(k)}}_{\infty}}$ on the y-axis. If you click on one of the points, the value of $(x_{0}, x_{1}, x_{2}, x_{3})$ will become visible. The legend is interactive as well, you can turn the lines off/on by simply clicking on the legend. 

The second plot "Errors"  shows the number of iteration on the x-axis and the relative error on the y-axis. You can also click on each point to see the absolute errors for each $(x_{0}, x_{1}, x_{2}, x_{3})$. The legend can also be turned on/off. 

<mark>Side Notes</mark>: In Convergence Criterion plot the first observation with the initial guess $x^{0} = (0,0,0,0).T$ is not plotted because the quotient equals to infinity. 

### Observations

We observe that Gauss-Seidel method with different initial guess converges to the solution with consistency to four decimal points in 6 iterations. From both plots above, we can observe, that the algorithm that uses the initial guess $x^{0} = (1,1,1,1).T$ converges slightly faster and starts stagnating - this behaviour is expected since this initial guess is closer to the actual solution. 



The second plot captures the relative errors: the relative errors have been chosen since they allow to see how large the error actualy is in relation to the correct value. The second plot corroborates with our results: it is easier to observe that between 2-4 iterations, for the initial guess $x^{0} = (1,1,1,1).T$ the relative error is smaller compared to  initial guess $x^{0} = (0,0,0,0).T$. We should also note, that this is supported by the theoretical result, that tells us that as the number of iterations increases, the error goes to zero.

We should also keep in mind that the convergence criterion is quite critical since it determines the effectiveness of the initial guess. Here we are said to make use of the L infinity-norm.



**Let's now examine the rate of convergence**

Let's first denote a few relevant ideas: 

We first should define a spectral radius: 

Given $ N \times N$ matrix M the *spectral radius* $\rho(\textbf{M})$ is defined as

\begin{equation*}
\rho(\textbf{M}) = \max|\lambda_{i}|, i = 1,2,...., n
\end{equation*}
where $\lambda$ is the eigenvalue of $\textbf{M}$.


Secondly, let's suppose that matrix $A$ can be written in the form

\begin{equation*}
M = L + D + U
\end{equation*}

where 
1. D is a matrix with **ONLY** diagonal elements of $M$
2. L is a strictly lower-triangular matrix 
3. U is strictly upper part of M

For Gauss-Seidel we have:

\begin{equation*}
IM = B^{-1}(A+C)
\end{equation*}

Convergence of the GS method is proven theoretically by showing that the spectral radius of the iteration matrix $\textbf{IM} < 1$ [<sup>5</sup>](#fn5).


In numerical analysis, we are often interested in determining the rate of convergence of the method. The spectral radius is shown to be a good measure of the rate of convergence. 
We say that $|\lambda_{1}| > |\lambda_{}|$ $(i = 2, ..., N)$, then 

\begin{equation*}
\textbf{v}^{(k)} = \alpha_{1}\lambda^{k}_{1}\textbf{e}_{1} + \sum_{i=2}^{N}\alpha_{i}\lambda_{i}^{k}\textbf{e}_{1} = \lambda_{i}^{k} \big[\alpha_{1}\textbf{e}_{1} + \sum_{i=2}^{N}\alpha_{i} \big (\frac{\lambda_{i}}{\lambda_{1}} \big )^{k} \textbf{e}_{i} \big ]
\end{equation*}
where $\textbf{e}_{i}$ are the eigenvectors. 

Given that $\frac{\lambda_{1}}{\lambda_{1}}$ $(i = 2, ..., N)$ for large $k$,

\begin{equation*}
\textbf{v}^{(k)} \simeq \alpha_{i}\lambda^{k}_{1}\textbf{e}_{1} ,
\end{equation*}
where $\textbf{e}_{1}$ is the first eigenvector. 

Hence, the error associated with $\textbf{x}^{(k)}$, *k*th vector in the sequence, is given by $\textbf{v}^{(k)}$ which varies approximately as a *k*th of the largest eigenvalue. In other words, it varies as the *k*th power of the spectral radius $\rho(\textbf{M}) = \max|\lambda_{1}|$. Therefore, the spectral radius is a good indication of the rate of convergence. If it is < 1, it also ensures that the sequence will converge [<sup>4</sup>](#fn4).







In [44]:
# Finding spectral radius
# T = (D-L)^{-1}U
UPPER = np.triu(A) - np.diag(A) * np.identity(4)
LOWER = np.tril(A) - np.diag(A) * np.identity(4)
DIAGONAL = np.diag(A) * np.identity(4)

In [45]:
max_eigenvalue = max(abs(np.linalg.eig(np.linalg.inv(DIAGONAL) @ (LOWER + UPPER))[0]))
print('Spectral radius:', max_eigenvalue)

Spectral radius: 0.42643661084234186


Thus, as $ k \rightarrow \infty$, the iterates converge to zero at a rate governed by the dominant eigenvalue.

***

#### Exercise 4. **SOR:** Using an accelarion factor of $\omega=1.1$, repeat part 3. above for $\textbf{x}^{(0)} = (0,0,0,0)$

In [47]:
# Exercise 4
x0 = np.zeros(4)
x_sor, norms_sor, _, _ = gauss_seidel(A,b, x0, omega=1.1)

Iteration: 1 
 Solution vector: [ 0.66        2.566      -1.07294     0.85649575] 
 Inf norm: inf
Iteration: 2 
 Solution vector: [ 1.1123068   1.99038796 -1.03425629  1.01360515] 
 Inf norm: 0.22432269875292288
Iteration: 3 
 Solution vector: [ 0.99524838  1.99297887 -0.99480477  1.00225005] 
 Inf norm: 0.05881186187694755
Iteration: 4 
 Solution vector: [ 0.99855989  2.00040261 -0.99991091  0.99962117] 
 Inf norm: 0.0037249485598974
Iteration: 5 
 Solution vector: [ 1.0001687   2.00009917 -1.00007679  0.99998642] 
 Inf norm: 0.0008042432684686864
Iteration: 6 
 Solution vector: [ 1.00001093  1.99998757 -0.99999759  1.00000682] 
 Inf norm: 7.887918876179545e-05


<ipython-input-14-2bf733c1a108>:42: RuntimeWarning: divide by zero encountered in double_scalars
  stopCond = np.linalg.norm(x - previous_x, ord=np.inf) / np.linalg.norm(previous_x, ord=np.inf)


Let's now calculate our $\omega$ based on the spectral radius that we have calculated above. We should note that there is no complete answer to how $\omega$ should be chosen - yet there are a number of well known results which can be used in certain cases.

First, theory: 


A *positive definite* matrix is a symmetric matrix with all *positive* eigenvalues. All $n$ eigenvalues $\lambda$ are real numbers. 

If A is positive define tridiagonal matrix, then the optimal choice for $\omega$ for GS method: 

\begin{equation*}
\omega = \frac{2}{ 1 + \sqrt{1-\rho(T_{g})}}
\end{equation*}


Our matrix $A$ is indeed positive definite, yet matrix $A$ is not sparse. In general, SOR method works exceptionally well for the solution of large sparse systems since 1) it allows to converge quicker and 2) it is wasteful to use general methods because most of the arithmetic operations devoted to solving the set of equations involve sero operands. 

We shall now find $\omega$ based on computer spectral radius and see whether we converge quicker.

In [48]:
2 / (1 + np.sqrt(1 - max_eigenvalue))

1.1380839026584906

In [49]:
x0 = np.zeros(4)
gauss_seidel(A,b, x0, omega=1.138)

Iteration: 1 
 Solution vector: [ 0.6828      2.6570024  -1.10483841  0.84271096] 
 Inf norm: inf
Iteration: 2 
 Solution vector: [ 1.14240169  1.96203656 -1.04016266  1.03219365] 
 Inf norm: 0.2615601102534797
Iteration: 3 
 Solution vector: [ 0.98516935  1.98955791 -0.98860677  1.00163413] 
 Inf norm: 0.08013731768441176
Iteration: 4 
 Solution vector: [ 0.99826522  2.00193305 -1.00077149  0.99883982] 
 Inf norm: 0.006582303219158766
Iteration: 5 
 Solution vector: [ 1.00063497  2.00007919 -1.00016107  1.0001034 ] 
 Inf norm: 0.0011837306207425273
Iteration: 6 
 Solution vector: [ 0.99995805  1.99993598 -0.99996374  1.00001821] 
 Inf norm: 0.0003384487494547429
Iteration: 7 
 Solution vector: [ 0.99999025  2.00000593 -1.00000004  0.99999495] 
 Inf norm: 3.497552888259114e-05


<ipython-input-14-2bf733c1a108>:42: RuntimeWarning: divide by zero encountered in double_scalars
  stopCond = np.linalg.norm(x - previous_x, ord=np.inf) / np.linalg.norm(previous_x, ord=np.inf)


(array([ 0.99999025,  2.00000593, -1.00000004,  0.99999495]),
 array([       inf, 0.26156011, 0.08013732, 0.0065823 , 0.00118373,
        0.00033845, 0.00003498]),
 array([[ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.6828    ,  2.6570024 , -1.10483841,  0.84271096],
        [ 1.14240169,  1.96203656, -1.04016266,  1.03219365],
        [ 0.98516935,  1.98955791, -0.98860677,  1.00163413],
        [ 0.99826522,  2.00193305, -1.00077149,  0.99883982],
        [ 1.00063497,  2.00007919, -1.00016107,  1.0001034 ],
        [ 0.99995805,  1.99993598, -0.99996374,  1.00001821],
        [ 0.99999025,  2.00000593, -1.00000004,  0.99999495]]),
 7)

We see that if $\omega = 1.138$ it takes +1 iteration to converge to the solution assuming that the consistency has not been changed. 

We suppose that if our system was indeed sparse, the SOR method would be more suited and would perform better.

***

### Include a report which discusses your results, together with interesting observations of accuracy and computational efficiency.

<mark>MARKED</mark> **Let's now calculate the results, plot them and investigate computational efficiency. If the plot down below is not generated, one should simply rerun the functions written below. Interactive plots are integrated with JS so they might not be visible once the notebook is disconnected from kernel.**

In [51]:
from IPython.display import clear_output
tolerance = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-9, 1e-10, 1e-11, 1e-12]
iterations_GS = []
iterations_SOR_given = []
iterations_SOR_calculated = []
for i in range(len(tolerance)):
    x0 = np.zeros(4)
    _, _, _, iterations = gauss_seidel(A, b, x0, tol = tolerance[i])
    iterations_GS.append(iterations)
    _, _, _, iterations = gauss_seidel(A, b, x0, tol = tolerance[i], omega = 1.1)
    iterations_SOR_given.append(iterations)
    _, _, _, iterations = gauss_seidel(A, b, x0, tol = tolerance[i], omega = 1.138)
    iterations_SOR_calculated.append(iterations)
    clear_output()

In [52]:
# Creating plotting objects
efficiency = figure(
                  x_axis_type="log", plot_height=300, plot_width=600,
                  title='Computational Efficiency',
                  x_axis_label='Tolerance', y_axis_label='Iterations',
                  toolbar_location=None)
efficiency.line(tolerance, iterations_GS, legend_label='Gauss-Seidel', line_color="blue", line_width=1, muted_alpha=0)
efficiency.line(tolerance, iterations_SOR_given, legend_label='SOR, omega = 1.1', line_color="orange", line_width=1, muted_alpha=0)
efficiency.line(tolerance, iterations_SOR_calculated, legend_label='SOR, omega = 1.138', line_color="green", line_width=1, muted_alpha=0)
efficiency.circle(x= tolerance,
           y=iterations_GS,
           color='royalblue',
             fill_color="white",
           selection_color='deepskyblue',
           nonselection_color='lightgray',
           legend_label='Gauss-Seidel',
            muted_alpha=0,
           nonselection_alpha=0.3)
efficiency.circle(x= tolerance,
           y=iterations_SOR_given,
           color='orange',
             fill_color="white",
           selection_color='deepskyblue',
           nonselection_color='lightgray',
           legend_label='SOR, omega = 1.1',
            muted_alpha=0,
           nonselection_alpha=0.3)
efficiency.circle(x= tolerance,
           y=iterations_SOR_calculated,
           color='green',
             fill_color="white",
           selection_color='deepskyblue',
           nonselection_color='lightgray',
           legend_label='SOR, omega = 1.138',
            muted_alpha=0,
           nonselection_alpha=0.3)

GlyphRenderer(id='2049', ...)

In [53]:
efficiency.legend.click_policy = 'hide'
efficiency.legend.click_policy = 'mute'

In [99]:
output_notebook()
show(efficiency)

Loading BokehJS ...

The following semilog plot examines the computational efficiency of three algorithms: the tolerance level is given on the x-axis and number of iterations it takes for the algorithm to converge is given as x-axis. $x^{0} = (0,0,0,0)$ is taken as initial guess. 

Direct method that involves constructing LU factorization is good for dense systems of small to moderate size, however as the coefficient matrix becomes large and sparse, it becomes wasteful as most of the arithmetic operations devoted to solving the set of equations involve zero operanda. One big advantage of iterative techniques is that we can control how accurate we want our solution, for example, if we want our solution to be $\varepsilon$-accurate (whether relative or absolute), we set the termination criteria accordignly. This saves a lot of computations! We now examine the computational efficiency of iterative techniques applied to our initial problem.

We note that when the tolerance level is set between $1e-12 \text{ and } 1e-8$, for all three algorithms it takes the same amount of iterations to converge, yet as the epsilon gets large, we shall observe that the number of iterations start differ. Firstly, let's examine the difference between Gauss-Seidel and SOR (omega = 1.1) methods: it takes the same amount of iterations for both to converge, except when the epsilon is set to $1e-7$ and $1e-6$. For those, GS method converges slightly faster (1 iteration less). Secondly, let's compare the results for SOR method with two $\omega$, set to 1.1 and 1.138 respectively. When the $\varepsilon < 1e-5$, we do not see any difference, however as epsilone gets large, it takes one iteration less for SOR method with $\omega = 1.1$. 

We note that the purpose of over-relaxation methods is to accelerate the converges of systems which are convergent by Gauss-Seidel[<sup>5</sup>](#fn5). Therefore, we shall conclude that our results are inconclusive, since we cannot observe a significant difference in computational efficiency. This analysis can be expanded if one takes a sparse system and compare efficiency results for aforementioned methods. 


In [85]:
%%time 
gauss_seidel(A, b, x0)
clear_output()

CPU times: user 3.87 ms, sys: 1.86 ms, total: 5.73 ms
Wall time: 4.35 ms


In [91]:
%%time 
gauss_seidel(A, b, x0, omega=1.1)
clear_output()


CPU times: user 4.91 ms, sys: 2.93 ms, total: 7.83 ms
Wall time: 5.38 ms


In [92]:
%%time 
gauss_seidel(A, b, x0, omega=1.138)
clear_output()

CPU times: user 4.41 ms, sys: 2.43 ms, total: 6.84 ms
Wall time: 4.8 ms


### References
<span id="fn1">[1] Lim, L.-H. (n.d.). Mathematical Computations, The University of Chicago. Retrieved March 31, 2021, from http://www.stat.uchicago.edu/~lekheng/courses/309f16/notes/lect16.pdf.</span> <br>
<span id="fn2">[2] Gabellini Francesca. (n.d.). Iterative Methods for Solving Linear Systems.</span> <br>
<span id="fn3">[3] Strang, G. (2019). Linear Algebra and Learning from Data. www.wellesleycambridge.com</span> <br>
<span id="fn4">[4] Olver, P. J. (n.d.). Numerical Analysis Lecture Notes 7. Iterative Methods for Linear Systems.</span> <br>
<span id="fn5">[5] Ahmad, R. (2021). Numerical Linear Algebra.</span> <br>